##### Version for Multiple obj points (2 or 3) and multiple shifts, generating a x,y rotating moving object prediction. 
##### To run: chose proper os.chdir and data_path. chose proper input_dim, and note other hyper settings. Chose desired relative_pos_list. Rest doesn't need to change necessarily.


In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split,Subset
from torch.nn.parameter import Parameter
from logger import Logger
import os
from utils_twopoints import *
from visualize import show2D_2objs, show2D_3objs, show3Dpred_2objs
import matplotlib.pyplot as plt

In [2]:
np.random.seed(190)
torch.manual_seed(120) ##set the random seed so when construct the 

#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_03 data')
#data_path='18_11_03sweepxyz'
#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_21 CCD XYZ sweep')
#data_path='FS_LD9by9_All_CCD sweep xyz_avg_win20_normalizedbymax'
os.chdir('/home/zyhuang/WD/Net3D/2018_11_21 CCD XYZ sweep/')
data_path='FS_4by4_CMOS_avg_win20_normalizedbymax'
scale_z=True #divide z coord by 33.3 to make it has same range as x,y
lr=1e-3
num_epoch=2000
logfreq=10
rotating_idx = [0,1,2,3,0,1,2,3,0,1,2]#rotation
num_trajectory = 9*4+2
xy_center_idx = []
while len(xy_center_idx) < num_trajectory:
    x = np.random.randint(low=2,high=9)#  integer in [2,8], only suitable for the current shape of object and spatial grid
    y = np.random.randint(low=2,high=9)#  only suitable for the current shape of object and spatial grid
    if (x,y) not in xy_center_idx:
        xy_center_idx.append((x,y))
bs_test=len(rotating_idx)*len(xy_center_idx) #specify test set size
bs_train=100
num_features=64
input_dim=4*4*2
#input_dim=32*32*2
nx,ny,nz=11,11,11
relative_pos_list=[[(0,0,0),(2,4,0)],[(0,0,0),(4,2,0)],[(4,0,0),(0,2,0)],[(2,0,0),(0,4,0)]] #for two objs, four shifts

Nshift=len(relative_pos_list)
Nobj=len(relative_pos_list[0])

#concatenate dataset of different orientation
ds_o1 = FSdataset_twopoints_v2(data_path,relative_pos=relative_pos_list[0],nx=nx,ny=ny,nz=nz,scale_z=scale_z)
ds_o2 = FSdataset_twopoints_v2(data_path,relative_pos=relative_pos_list[1],nx=nx,ny=ny,nz=nz,scale_z=scale_z)
ds_o3 = FSdataset_twopoints_v2(data_path,relative_pos=relative_pos_list[2],nx=nx,ny=ny,nz=nz,scale_z=scale_z)
ds_o4 = FSdataset_twopoints_v2(data_path,relative_pos=relative_pos_list[3],nx=nx,ny=ny,nz=nz,scale_z=scale_z)
ds_Full = ds_o1+ds_o2+ds_o3+ds_o4

test_idx = []
#append idx for trajectory at each center location
for xy in xy_center_idx:
    test_idx.extend(genRotation_data_idx_multi_orientation(xy[0],xy[1],[2,4,4,2],[4,2,2,4],nx=11,ny=11,nz=11,single_orientation_ds_list=[ds_o1,ds_o2,ds_o3,ds_o4],rotating_idx=rotating_idx))
# Split train/test set
train_idx = np.arange(len(ds_Full)).tolist()
for i in test_idx:
    train_idx.remove(i)

ds_train = Subset(ds_Full,train_idx)
ds_test = Subset(ds_Full,test_idx)

train_loader=DataLoader(ds_train, batch_size=bs_train,shuffle=True)
test_loader=DataLoader(ds_test, batch_size=bs_test,shuffle=False)

save_name = '4by4_avg20'+ str(Nobj)+'_objs_singlenetxyz_Set_loss_'+str(Nshift)+'shifts_scale_z_'+str(scale_z)+'/'+"".join([s for s in str(relative_pos_list) if s.isdigit()])\
    +'/depthnet4_Nobj_xyz('+str(num_features)+'features)'+'_adam_lr_'+str(lr)+'_bs_train_'+str(bs_train)+'_bs_test_'+str(bs_test)+'_numepoch_'+str(num_epoch)+ '/rot_'\
    + 'num_traj_' + str(num_trajectory)
logger = Logger('./logs_paper_data/'+ save_name) ##set the logger for tensorboard and save the log file to ./logs/Adam 
#dtype = torch.FloatTensor ##run on cpu
dtype = torch.cuda.FloatTensor 

net=depthnet4_Nobj_xyz(input_dim,num_features,Nobj=Nobj)
net.type(dtype)


criterion=Set_loss_2obj()   
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
36/252

In [ ]:
loss_history=[]
test_loss_history=[]
for epoch in range(num_epoch):
    if np.mod(epoch,logfreq)==0: 
        print("Current epoch number%d" %epoch) 
    for idx,data in enumerate(train_loader,0):
        net.train()
        if Nobj == 2:
            FS_bat,xyz1_true,xyz2_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype))
            xyz1_pred,xyz2_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6]
            loss=criterion(xyz1_pred,xyz2_pred,xyz1_true,xyz2_true)
        elif Nobj == 3:
            FS_bat,xyz1_true,xyz2_true,xyz3_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype)),Variable(data['xyz'][:,6:9].type(dtype))
            xyz1_pred,xyz2_pred,xyz3_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6],net(FS_bat)[:,6:9]
            loss=criterion(xyz1_pred,xyz2_pred,xyz3_pred,xyz1_true,xyz2_true,xyz3_true)            
        loss_history.append(loss.data[0])
        if np.mod(epoch,logfreq)==0: 
            print('loss is %3f'%(loss.data[0]))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if np.mod(epoch,logfreq)==0:  
        net.eval()
        data=next(iter(test_loader))
        if Nobj ==2:
            FS_bat,xyz1_true,xyz2_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype))
            xyz1_pred,xyz2_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6]
            test_loss=criterion(xyz1_pred,xyz2_pred,xyz1_true,xyz2_true)
        elif Nobj ==3:
            FS_bat,xyz1_true,xyz2_true,xyz3_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype)),Variable(data['xyz'][:,6:9].type(dtype))
            xyz1_pred,xyz2_pred,xyz3_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6],net(FS_bat)[:,6:9]
            test_loss=criterion(xyz1_pred,xyz2_pred,xyz3_pred,xyz1_true,xyz2_true,xyz3_true)
            
        test_loss_history.append(test_loss.data[0])       
        print('Test Loss is %3f' %(test_loss.data[0]))#for multiple test sample
        
        step=epoch+1
        info = {
        'training_loss': loss.data[0],
        'val_loss':test_loss.data[0],
        }
        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)

torch.save(net.state_dict(), os.path.join('logs_paper_data',save_name, 'model.pth'))

In [ ]:
plt.figure()
plt.semilogy(loss_history[::len(train_loader)],label='training_loss')#training_loss
plt.legend()
plt.title('Training loss vs epoch number')
plt.savefig('logs_paper_data/'+save_name+'/Train loss')

plt.figure()
plt.semilogy(np.arange(0,epoch,logfreq),test_loss_history[:],label='test_loss')
plt.legend()
plt.title('Test loss vs epoch number')
plt.savefig('logs_paper_data/'+save_name+'/Test loss')

In [ ]:
#this part of code is added , previsouly the test loss is not the final model test loss since np.mod(epoch,logfreq)==0 is false for last epoch 
net.eval()

data=next(iter(test_loader))
if Nobj ==2:
    FS_bat,xyz1_true,xyz2_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype))
    xyz1_pred,xyz2_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6]
    test_loss=criterion(xyz1_pred,xyz2_pred,xyz1_true,xyz2_true)
elif Nobj ==3:
    FS_bat,xyz1_true,xyz2_true,xyz3_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype)),Variable(data['xyz'][:,6:9].type(dtype))
    xyz1_pred,xyz2_pred,xyz3_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6],net(FS_bat)[:,6:9]
    test_loss=criterion(xyz1_pred,xyz2_pred,xyz3_pred,xyz1_true,xyz2_true,xyz3_true)
text_file = open('logs_paper_data/'+save_name+"/Loss_final.txt", "w")
text_file.write('loss is %.4f(%.3f mm)\n'%(loss.data[0],np.sqrt(loss.data[0])))
text_file.write('Test Loss is %.4f(%.3f mm).' %(test_loss.data[0],np.sqrt(test_loss.data[0])))
text_file.close()
##

In [ ]:
test_bat=next(iter(test_loader))
FS_bat,trueXYZobjs_bat=test_bat['FS'],test_bat['xyz'][:,:]
#predXYZobjs_bat=torch.cat((net_x(Variable(FS_bat.type(dtype))),net_y(Variable(FS_bat.type(dtype))),net_z(Variable(FS_bat.type(dtype)))),dim=1).data.cpu()[:,[0,2,4,1,3,5]]
predXYZobjs_bat=net(Variable(FS_bat).type(dtype)).data.cpu()
print(predXYZobjs_bat)
print(trueXYZobjs_bat)
globals()['show2D_'+str(Nobj)+'objs'](trueXYZobjs_bat,predXYZobjs_bat,howmany=11,save_name='logs_paper_data/'+save_name+'/2Ds')   

In [ ]:
%matplotlib auto
show3Dpred_2objs(trueXYZobjs_bat,predXYZobjs_bat,howmany=8,save_name=None)

In [ ]:
from scipy import io
data={'trueXYZ':trueXYZobjs_bat.cpu().numpy(), 'PredictedXYZ':predXYZobjs_bat.cpu().numpy()}
io.savemat('logs_paper_data/'+save_name+"/testData.mat",data)

## Test Time serialization part

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split,Subset
from torch.nn.parameter import Parameter
from logger import Logger
import os
from utils_twopoints import *
from visualize import show2D_2objs, show2D_3objs, show3Dpred_2objs
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(190)
torch.manual_seed(120) ##set the random seed so when construct the 

#os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_03 data')
#data_path='18_11_03sweepxyz'
os.chdir('E:/Academic/MATLAB/Dehui_FS/2018_11_21 CCD XYZ sweep')
data_path='FS_LD32by32_All_CCD sweep xyz_avg_win4_normalizedbymax'
scale_z=True #divide z coord by 33.3 to make it has same range as x,y
rotating_idx = [0,1,2,3,0,1,2,3,0,1,2]#rotation
num_trajectory = 9
xy_center_idx = []
while len(xy_center_idx) < num_trajectory:
    x = np.random.randint(low=2,high=9)#  integer in [2,8], only suitable for the current shape of object and spatial grid
    y = np.random.randint(low=2,high=9)#  only suitable for the current shape of object and spatial grid
    if (x,y) not in xy_center_idx:
        xy_center_idx.append((x,y))
bs_test=len(rotating_idx)*len(xy_center_idx) #specify test set size
bs_train=100
num_features=64
#input_dim=4*4*2
input_dim=32*32*2
nx,ny,nz=11,11,11
relative_pos_list=[[(0,0,0),(2,4,0)],[(0,0,0),(4,2,0)],[(4,0,0),(0,2,0)],[(2,0,0),(0,4,0)]] #for two objs, four shifts

Nshift=len(relative_pos_list)
Nobj=len(relative_pos_list[0])

#concatenate dataset of different orientation
ds_o1 = FSdataset_twopoints_v2(data_path,relative_pos=relative_pos_list[0],nx=nx,ny=ny,nz=nz,scale_z=scale_z)
ds_o2 = FSdataset_twopoints_v2(data_path,relative_pos=relative_pos_list[1],nx=nx,ny=ny,nz=nz,scale_z=scale_z)
ds_o3 = FSdataset_twopoints_v2(data_path,relative_pos=relative_pos_list[2],nx=nx,ny=ny,nz=nz,scale_z=scale_z)
ds_o4 = FSdataset_twopoints_v2(data_path,relative_pos=relative_pos_list[3],nx=nx,ny=ny,nz=nz,scale_z=scale_z)
ds_Full = ds_o1+ds_o2+ds_o3+ds_o4

test_idx = []
#append idx for trajectory at each center location
for xy in xy_center_idx:
    test_idx.extend(genRotation_data_idx_multi_orientation(xy[0],xy[1],[2,4,4,2],[4,2,2,4],nx=11,ny=11,nz=11,single_orientation_ds_list=[ds_o1,ds_o2,ds_o3,ds_o4],rotating_idx=rotating_idx))
# Split train/test set
train_idx = np.arange(len(ds_Full)).tolist()
for i in test_idx:
    train_idx.remove(i)

ds_train = Subset(ds_Full,train_idx)
ds_test = Subset(ds_Full,test_idx)

train_loader=DataLoader(ds_train, batch_size=bs_train,shuffle=True)
test_loader=DataLoader(ds_test, batch_size=bs_test,shuffle=False)

model_path = 'model.pth'

#dtype = torch.FloatTensor ##run on cpu
dtype = torch.cuda.FloatTensor 

net=depthnet4_Nobj_xyz(input_dim,num_features,Nobj=Nobj)
net.type(dtype)
net.load_state_dict(torch.load(model_path))
criterion=Set_loss_2obj()   


In [ ]:
#this part of code is added , previsouly the test loss is not the final model test loss since np.mod(epoch,logfreq)==0 is false for last epoch 
net.eval()

data=next(iter(test_loader))
if Nobj ==2:
    FS_bat,xyz1_true,xyz2_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype))
    xyz1_pred,xyz2_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6]
    test_loss=criterion(xyz1_pred,xyz2_pred,xyz1_true,xyz2_true)
    print("Test loss is %.4f" %test_loss)
elif Nobj ==3:
    FS_bat,xyz1_true,xyz2_true,xyz3_true=Variable(data['FS'].type(dtype)),Variable(data['xyz'][:,0:3].type(dtype)),Variable(data['xyz'][:,3:6].type(dtype)),Variable(data['xyz'][:,6:9].type(dtype))
    xyz1_pred,xyz2_pred,xyz3_pred=net(FS_bat)[:,0:3],net(FS_bat)[:,3:6],net(FS_bat)[:,6:9]
    test_loss=criterion(xyz1_pred,xyz2_pred,xyz3_pred,xyz1_true,xyz2_true,xyz3_true)
    print("Test loss is %.4f" %test_loss)
##

In [ ]:
test_bat=next(iter(test_loader))
FS_bat,trueXYZobjs_bat=test_bat['FS'],test_bat['xyz'][:,:]
#predXYZobjs_bat=torch.cat((net_x(Variable(FS_bat.type(dtype))),net_y(Variable(FS_bat.type(dtype))),net_z(Variable(FS_bat.type(dtype)))),dim=1).data.cpu()[:,[0,2,4,1,3,5]]
predXYZobjs_bat=net(Variable(FS_bat).type(dtype)).data.cpu()
print(predXYZobjs_bat)
print(trueXYZobjs_bat)
globals()['show2D_'+str(Nobj)+'objs'](trueXYZobjs_bat,predXYZobjs_bat,howmany=11)   

In [ ]:
%matplotlib auto
show3Dpred_2objs(trueXYZobjs_bat,predXYZobjs_bat,howmany=8,save_name=None)

In [ ]:
from scipy import io
data={'trueXYZ':trueXYZobjs_bat.cpu().numpy(), 'PredictedXYZ':predXYZobjs_bat.cpu().numpy()}
io.savemat('CMOS32by32.mat',data)